In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [3]:
summoner_name = "쪼렙이다말로하자"

match = pd.read_json("data/data_summoner_match_data_{name}.json".format(name=summoner_name))
timeline = pd.read_json("data/data_summoner_match_timeline_data_{name}.json".format(name=summoner_name))
with open("data/data_summoner_{}.json".format(summoner_name), 'r') as f:
    summoner = json.load(f)

# 경기 관련 데이터

In [4]:
match_length = len(match["info"])
match_length

200

In [5]:
info = match["info"]

participants = []
teams = []
for i in range(match_length):
    participants.append(info[i]["participants"])
    teams.append(info[i]["teams"])
    del(info[i]["participants"])
    del(info[i]["teams"])

In [6]:
df_info = pd.json_normalize(info)
df_info_1 = df_info[["gameDuration", "gameMode", "gameType", "gameVersion", "mapId"]].copy()    # 원본데이터는 유지
display(df_info_1.head(10))

gameDuration gameMode      gameType     gameVersion  mapId
0       1371736  CLASSIC  MATCHED_GAME  11.19.398.9466     11
1       1542991  CLASSIC  MATCHED_GAME  11.19.398.9466     11
2       1468270  CLASSIC  MATCHED_GAME  11.19.398.9466     11
3       1963800  CLASSIC  MATCHED_GAME  11.19.398.9466     11
4       1634952  CLASSIC  MATCHED_GAME  11.19.398.9466     11
5       1932577  CLASSIC  MATCHED_GAME  11.19.398.9466     11
6       1478309  CLASSIC  MATCHED_GAME  11.19.398.9466     11
7       1207067  CLASSIC  MATCHED_GAME  11.19.398.2521     11
8        914139  CLASSIC  MATCHED_GAME  11.19.398.2521     11
9       1516101  CLASSIC  MATCHED_GAME  11.18.395.7538     11

In [7]:
df_participants = pd.json_normalize(participants)
display(df_participants.head(1))

0  \
0  {'assists': 3, 'baronKills': 0, 'bountyLevel':...   

                                                   1  \
0  {'assists': 8, 'baronKills': 1, 'bountyLevel':...   

                                                   2  \
0  {'assists': 8, 'baronKills': 0, 'bountyLevel':...   

                                                   3  \
0  {'assists': 9, 'baronKills': 0, 'bountyLevel':...   

                                                   4  \
0  {'assists': 9, 'baronKills': 0, 'bountyLevel':...   

                                                   5  \
0  {'assists': 2, 'baronKills': 0, 'bountyLevel':...   

                                                   6  \
0  {'assists': 4, 'baronKills': 0, 'bountyLevel':...   

                                                   7  \
0  {'assists': 3, 'baronKills': 0, 'bountyLevel':...   

                                                   8  \
0  {'assists': 2, 'baronKills': 0, 'bountyLevel':...   

                                                   9  
0  {'assists': 4, 'baronKills': 0, 'bountyLevel':...

In [8]:
# 표본의 게임 속 index 구하기
summoner_index = df_participants.apply(lambda row: np.where(row.map(lambda col: col["puuid"] == summoner["puuid"]))[0][0], axis=1)
# 표본의 게임 속 teamId 구하기
summoner_team = ["100" if summoner_index[x]<5 else "200" for x in range(match_length)]
# info_1 데이터프레임에 저장
df_info_1["index"] = summoner_index
df_info_1["team"] = summoner_team
display(df_info_1.head(10))

gameDuration gameMode      gameType     gameVersion  mapId  index team
0       1371736  CLASSIC  MATCHED_GAME  11.19.398.9466     11      2  100
1       1542991  CLASSIC  MATCHED_GAME  11.19.398.9466     11      9  200
2       1468270  CLASSIC  MATCHED_GAME  11.19.398.9466     11      7  200
3       1963800  CLASSIC  MATCHED_GAME  11.19.398.9466     11      9  200
4       1634952  CLASSIC  MATCHED_GAME  11.19.398.9466     11      2  100
5       1932577  CLASSIC  MATCHED_GAME  11.19.398.9466     11      7  200
6       1478309  CLASSIC  MATCHED_GAME  11.19.398.9466     11      9  200
7       1207067  CLASSIC  MATCHED_GAME  11.19.398.2521     11      7  200
8        914139  CLASSIC  MATCHED_GAME  11.19.398.2521     11      7  200
9       1516101  CLASSIC  MATCHED_GAME  11.18.395.7538     11      4  100

In [9]:
df_teams = pd.json_normalize(teams)
df_teams.columns = ["100", "200"]   # teamId
display(df_teams.head(5))

100  \
0  {'bans': [{'championId': -1, 'pickTurn': 1}, {...   
1  {'bans': [{'championId': 238, 'pickTurn': 1}, ...   
2  {'bans': [{'championId': 23, 'pickTurn': 1}, {...   
3  {'bans': [{'championId': 412, 'pickTurn': 1}, ...   
4  {'bans': [{'championId': 711, 'pickTurn': 1}, ...   

                                                 200  
0  {'bans': [{'championId': -1, 'pickTurn': 6}, {...  
1  {'bans': [{'championId': 126, 'pickTurn': 6}, ...  
2  {'bans': [{'championId': 39, 'pickTurn': 6}, {...  
3  {'bans': [{'championId': 412, 'pickTurn': 6}, ...  
4  {'bans': [{'championId': 64, 'pickTurn': 6}, {...

In [10]:
# 이긴 팀의 teamId 구하기
teams_win = df_teams.apply(lambda row: "100" if row.map(lambda col: col["win"])[0] else "200", axis=1)
df_teams_1 = df_teams.copy()
df_teams_1["win"] = teams_win

## 데이터 저장

In [11]:
df_info_1.to_csv("data/{}/info.csv".format(summoner_name), index=False)
df_participants.to_json("data/{}/participants.json".format(summoner_name))
df_teams_1.to_json("data/{}/teams.json".format(summoner_name))

# 인베이드 관련 데이터

In [12]:
match_invade = 0

## 이벤트 분석

In [37]:
12200 // 4000

3

In [39]:
is_invade = [False]*match_length
invade_timestamp = [0]*match_length           
invade_position = [0]*match_length           
invade_kill_blue = [0]*match_length           # invade시 발생한 블루팀의 킬 수
invade_kill_red = [0]*match_length            # invade시 발생한 레드팀의 킬 수
invade_win_team = [0]*match_length            # invade가 발생한 게임에서 이긴 팀의 teamId
total_gold_blue_3_min = [0]*match_length
total_gold_red_3_min = [0]*match_length
gold_gap = [0]*match_length

for match_index in range(match_length):
    match_time_length = len(timeline.iloc[match_index].info['frames'])
    n_invade_position = np.array([0 for _ in range(64)]).reshape(8,8)
    
    events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
    participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

    # 1~2분대의 kill event
    if len(events) < 3: 
        print("다시하기 발생!")
        continue    # 게임이 3분 이내로 끝났을 경우를 제외 
    for t in range(1,3):  # 1 ~ 2 분 사이
        for e in events[t]:    
            if e["type"] == "CHAMPION_KILL":
                # print(e)
                if is_invade[match_index] == False: 
                    is_invade[match_index] = True
                    match_invade += 1
                if invade_timestamp[match_index] == 0: invade_timestamp[match_index] = e['timestamp']
                if e['killerId'] <= 5:
                    invade_kill_blue[match_index] += 1
                elif e['killerId'] > 5:
                    invade_kill_red[match_index] += 1
                p_x, p_y = e['position']['x'], e['position']['y']
                n_invade_position[(16000-p_y)//2000][p_x//2000] += 1

    if is_invade[match_index]:
        print(n_invade_position)

        for i in range(1,6):
            total_gold_blue_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        for i in range(6,11):
            total_gold_red_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        gold_gap[match_index] = total_gold_blue_3_min[match_index] - total_gold_red_3_min[match_index]
        
        # use df_teams
        invade_win_team[match_index] = 100 if df_teams["100"][match_index]["win"] else 200
        
        print("--------------------------------------------------")
        print("invade detected: %d" % match_index)
        print("position:", invade_position_x[match_index], invade_position_y[match_index])
        print("timestamp(minutes): %.2f" % (invade_timestamp[match_index]/60000))
        print("< kills >\n  blue : %d , red : %d"% (invade_kill_blue[match_index], invade_kill_red[match_index]))
        print("< total_gold_3_min >\n  blue: %d , red: %d"% (total_gold_blue_3_min[match_index], total_gold_red_3_min[match_index]))
        print("< gold_gap >\n  %d"% gold_gap[match_index])
        print("< win >\n  %s"% ("blue" if invade_win_team[match_index] == 100 else "red"))

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]]
--------------------------------------------------
invade detected: 3
position: 12974 1333
timestamp(minutes): 1.87
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 4419 , red: 5324
< gold_gap >
  -905
< win >
  blue
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 3 0 0]]
--------------------------------------------------
invade detected: 8
position: 6774 770
timestamp(minutes): 1.59
< kills >
  blue : 3 , red : 0
< total_gold_3_min >
  blue: 5579 , red: 4008
< gold_gap >
  1571
< win >
  blue
[[0 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]]
--------------------------------------------------
invade detected: 9
position: 8769 8445
timestamp(minutes): 1.75
< kills >
  blue : 2 , red : 1
< total_gold_3_min >
  blue: 5063 , red: 4624
< gold_gap >
  439
< win >
  blue
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]]
--------------------------------------------------
invade detected: 13
position: 8926 7976
timestamp(minutes): 1.56
< kills >
  blue : 1 , r

In [30]:
df_invade = pd.DataFrame([
    is_invade,
    invade_timestamp,
    invade_position_x,
    invade_position_y,
    invade_kill_blue,
    invade_kill_red,
    invade_win_team,
    total_gold_blue_3_min,
    total_gold_red_3_min,
    gold_gap,
    df_info_1["gameMode"]]).transpose()

df_invade.columns = [
    "is_invade",
    "invade_timestamp",
    "invade_position_x",
    "invade_position_y",
    "invade_kill_blue",
    "invade_kill_red",
    "invade_win_team",
    "total_gold_blue_3_min",
    "total_gold_red_3_min",
    "gold_gap",
    "gameMode"]

display(df_invade[(df_invade.is_invade==True) & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])

# 인베가 일어났고, 블루가 레드보다 킬을 많이 딴 경기에서 블루가 이길 확률 
print("%.2f%%"%(
    len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red) 
        & (df_invade.invade_win_team == 100)]) 
    / len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])
    * 100)
)


is_invade invade_timestamp invade_position_x invade_position_y  \
8        True            95344              6774               770   
9        True           105263              8769              8445   
13       True            93590              8926              7976   
17       True            66227             11881              5705   
27       True           103481             11588              5910   
36       True            61481              3224             13933   
43       True            98819             10564              7453   
44       True           101375             13065              7414   
47       True           112460              6756              7088   
48       True            54482              8674              8662   
55       True            66861              5561              8806   
58       True            49414             10909              4328   
85       True           113900              7492              7951   
91       True            39432              5629              6431   
107      True            51829              8519              5159   
125      True           100800             12147              2228   
142      True            50990              4598             13826   
183      True            38312              8416              5418   

    invade_kill_blue invade_kill_red invade_win_team total_gold_blue_3_min  \
8                  3               0             100                  5579   
9                  2               1             100                  5063   
13                 1               0             100                  4798   
17                 2               0             100                  5910   
27                 1               0             100                  4908   
36                 1               0             200                  4858   
43                 2               0             100                  5600   
44                 1               0             100                  5365   
47                 1               0             100                  4496   
48                 1               0             100                  5442   
55                 1               0             200                  4551   
58                 3               0             200                  5376   
85                 1               0             100                  5004   
91                 1               0             100                  4740   
107                1               0             200                  5340   
125                1               0             100                  5360   
142                1               0             100                  4974   
183                1               0             100                  4812   

    total_gold_red_3_min gold_gap gameMode  
8                   4008     1571  CLASSIC  
9                   4624      439  CLASSIC  
13                  4289      509  CLASSIC  
17                  4207     1703  CLASSIC  
27                  4408      500  CLASSIC  
36                  4474      384  CLASSIC  
43                  4562     1038  CLASSIC  
44                  3741     1624  CLASSIC  
47                  3976      520  CLASSIC  
48                  4641      801  CLASSIC  
55                  5887    -1336  CLASSIC  
58                  4169     1207  CLASSIC  
85                  4264      740  CLASSIC  
91                  3857      883  CLASSIC  
107                 4609      731  CLASSIC  
125                 4129     1231  CLASSIC  
142                 4524      450  CLASSIC  
183                 4115      697  CLASSIC

77.78%


In [31]:
match_invade

280

In [32]:
df_invade.to_csv("data/{}/invade.csv".format(summoner_name), index=False)